In [10]:
from transformers import BertModel, BertForMaskedLM, BertConfig, BertTokenizer

In [28]:
from pyknp import Juman
import torch
import re
from tqdm import tqdm_notebook

In [39]:
import sys
sys.path.append("src")

In [40]:
import preprocess

In [42]:
pre = preprocess.Preprocess()
pre.enable_data["NormalizedText"] = \
    [{key:pre.normalize(text) for key, text in texts.items()}
     for texts in pre.enable_data["Text"]]

FileNotFoundError: [Errno 2] File b'..\\data\\raw\\data.csv' does not exist: b'..\\data\\raw\\data.csv'

In [ ]:
vocab_path="data/pretrained/vocab.txt"
config_path = 'data/pretrained/config.json'
model_path = 'data/pretrained/pytorch_model.bin'
bert_tokenizer = BertTokenizer(vocab_path, do_lower_case=False, do_basic_tokenize=False)

In [13]:
config = BertConfig.from_json_file(config_path)
model = BertModel.from_pretrained(model_path, config=config)
model.eval();

In [14]:
bert_tokenizer.tokenize("はれ")

['は', '##れ']

In [17]:
text = "今日は良い天気なので公演で散歩したい"

In [19]:
jumanpp = Juman()

In [26]:
def sentence_embedding(text: str):
    text = re.sub(r" ", r"　", text)
    result = jumanpp.analysis(text)
    tokenized_text =[mrph.midasi for mrph in result.mrph_list()]

    bert_tokens = bert_tokenizer.tokenize(" ".join(tokenized_text))
    ids = bert_tokenizer.convert_tokens_to_ids(["[CLS]"] + bert_tokens[:126] + ["[SEP]"])
    tokens_tensor = torch.tensor(ids).reshape(1, -1)

    with torch.no_grad():
        all_encoder_layers, _ = model(tokens_tensor)

    embedding = all_encoder_layers.numpy()[0][-1]

    return embedding

In [27]:
def vectorize_bert(text_dict):
    vector = {}
    for key, text in text_dict.items():
        vector["key"] = sentence_embedding(text)
        
    global video_idx    
    video_idx += 1
    print(f"No.{video_idx} is vectorized.")
    return vector

In [ ]:
for idx, text_dict in tqdm_notebook(enumerate(pre.enable_data["NormalizedText"])):
    vector = vectorize_bert(text_dict)
    np.save(filename, vector)